# Understanding One-Time Pad

***Tasks:***

1. *Research the theoretical basis of the one-time pad, including its requirements and operational principles. Describe one-time pad conditions clearly.*

# One-Time Pad Implementation

***Tasks:*** *the encryption and decryption process between two parties, Alice and Bob.*

1. **Alice's Program** 
    - *Should prompt for a message input (plaintext), then display the ciphertext, and save both the ciphertext (in hex) and the key (in hex) in separate files.*

2. **Bob's Program**
    - *Should read the key and ciphertext from their respective files and display the decrypted plaintext.*

# Implementing Many-Time Pad

***Tasks:*** *Modify the one-time pad implementation to encrypt multiple messages with the same key, simulating a many-time pad scenario. The purpose of this problem is to see if there are any recognizable patterns by observing the outputs. You can gain insights by changing the plaintexts or the key to verify your findings. These findings would be useful in the next problem.*

1. *The program should encrypt a list of 10 predefined plaintext messages with a single key, saving the plaintexts, key, and ciphertexts (all in hex) into a file. You can select 10 of your favorite messages. Assume the key is long enough to do encryption to all the 10 messages.*

# Cryptanalysis of Many-Time Pad

***Tasks:*** *Develop a strategy to decrypt messages encrypted with a many-time pad. More specifically, assume Eva has collected the 10 ciphertexts and she knew they are generated by the same key. In addition, all the plaintexts are in English. Space, comma, period, and question mark are being used in the plaintext, but no other special characters are allowed. Eva wants to decrypt the last message (target message).*

***Collected ciphertexts:***

1) 71fe1ace4389087266117cd7c98c4182851b3acff3b086e3f83f94d6eb05c4ba85d8e1fa14f11d1c3b568ff6cff5c09c5d67ef5c9c71b7eeb3d45a5154ab17b83e071ce9d8988adb4afedf46a840
2) 71fe1ace559a1e7266117cd7ce8745d7be2e74c3f0f68eeef57e8884e607debf81dfa0f012f95819681ae7f29fe4839b5175ef5e8760bef0b9d44b504eba12b22f5404f89dd085d550a48865a14f9b15a94dabe609ca2df2cccf210cefdb1af5389719795e1f0179cb77c5c456954d88f3
3) 72fe069c51c81a20775928c7879d4fd2a93c3acff3f69fe5fe2e9493a303d9ea98c4e5b60ae40a146058e7c787fbd09a1474e25dc865b5e6af865d4a40a61bfd384e06e0cfc1ccd356ff8853ac4389
05fa5fe3fd41cb3bbc8ac9
4) 67e543885b9a5b2267177084cf8453ccb8633ad7fdb39de5b13f8a93a304d6bf8bc4f4ef5def110b6f56a3e186e2c68c1470ef5c9c2ffbd6a291571e40ba1afd3b4b1fe0c4cbccc15df5dc07b043da01fa6ae4fd158f37b3c0cd
5) 71fe029a148c1236320d7192878a59cfbc3a6ec5e7f68befb13196d6ea1ec4ea81d9e3fe50ea0f196d02a2f7cfe2c29c5577e35d8630baf6ea80465b01aa1abc394f57a1f4ccccda59ff8846e44b8805bb5cabe608c231f2dec8364ae7d90ab4358c5c3a421b06

In [12]:
import string
from collections import Counter
from Crypto.Util.strxor import strxor


# Allowed plaintext characters
ALLOWED = set(string.ascii_letters + " ,.?-01")

# Very simple English scoring
LETTER_FREQ_ORDER = " etaoinshrdlu"


def score_english(chars):
    """
    Higher score = more English-like.
    """
    score = 0
    counts = Counter(chars)

    # Reward spaces
    score += counts[" "] * 3

    # Reward common letters
    for c in LETTER_FREQ_ORDER:
        score += counts.get(c, 0) * 2

    # Penalize too much punctuation
    score -= counts[","] * 0.5
    score -= counts["?"] * 0.5
    score -= counts["."] * 0.5

    return score


# def recover_key(ciphertexts):
#     key = []
#     n = len(ciphertexts[0])

#     for j in range(n):
#         best_score = float("-inf")
#         best_key_byte = None

#         for k in range(256):
#             decrypted_chars = []

#             valid = True
#             for ct in ciphertexts:
#                 p = ct[j] ^ k
#                 if chr(p) not in ALLOWED:
#                     valid = False
#                     break
#                 decrypted_chars.append(chr(p))

#             if not valid:
#                 continue

#             score = score_english(decrypted_chars)

#             if score > best_score:
#                 best_score = score
#                 best_key_byte = k

#         key.append(best_key_byte)

#     return bytes(key)


# ---- Usage ----

# cts_hex = [
#     "71fe1ace4389087266117cd7c98c4182851b3acff3b086e3f83f94d6eb05c4ba85d8e1fa14f11d1c3b568ff6cff5c09c5d67ef5c9c71b7eeb3d45a5154ab17b83e071ce9d8988adb4afedf46a840",
#     "71fe1ace559a1e7266117cd7ce8745d7be2e74c3f0f68eeef57e8884e607debf81dfa0f012f95819681ae7f29fe4839b5175ef5e8760bef0b9d44b504eba12b22f5404f89dd085d550a48865a14f9b15a94dabe609ca2df2cccf210cefdb1af5389719795e1f0179cb77c5c456954d88f3",
#     "72fe069c51c81a20775928c7879d4fd2a93c3acff3f69fe5fe2e9493a303d9ea98c4e5b60ae40a146058e7c787fbd09a1474e25dc865b5e6af865d4a40a61bfd384e06e0cfc1ccd356ff8853ac438905fa5fe3fd41cb3bbc8ac9",
#     "67e543885b9a5b2267177084cf8453ccb8633ad7fdb39de5b13f8a93a304d6bf8bc4f4ef5def110b6f56a3e186e2c68c1470ef5c9c2ffbd6a291571e40ba1afd3b4b1fe0c4cbccc15df5dc07b043da01fa6ae4fd158f37b3c0cd",
#     "71fe029a148c1236320d7192878a59cfbc3a6ec5e7f68befb13196d6ea1ec4ea81d9e3fe50ea0f196d02a2f7cfe2c29c5577e35d8630baf6ea80465b01aa1abc394f57a1f4ccccda59ff8846e44b8805bb5cabe608c231f2dec8364ae7d90ab4358c5c3a421b06",
#     "6ef914ce5989152b321a769ad79c42c7be6f6ad2fab19de1fc339d84f04ad3a589dfa0ff09ab0c196f13e7e780b4c097556ded57c871fbeea393464a01aa0ab1381848cfd2d6898918efc046b00b",
#     "71fe1ace4389087266117cd7c4865bd2b93b7fd2b5a58ce9f4308c9ff01e97ab82cbf2ef5dfc101d6a56b3fb8ab4d08b4167ef5c9c30b8f0ab97455b45e81efd364605e49ddb83df48eedc42b60c",
#     "71fe029a148c1437615978d7c58854dbec2c75cde5a39be5e37e9b97ef0697a285dfa0f01cff101d764983f29bf5",
#     "71fe1ace50875b31730d6ad7cb8640c7ec3c73d4e1bf81e7b13796d6e518d8a4988ceff05dff101d2415a8fe9fe1d79a4623eb5e8430bfe3b3d442514faf40fd18420be0c8cb89924cf3cd5ee44895",
#     "71fe029a1483123c76597691878459cca9363ac4faf68ceffc2e8d82e61897b98fc5e5f809e20b0c7756b2e08aab83bc5560e257",
#     "71fe0680149d083b7c1e3996879a42d0a92e7780f6bf9fe8f42cd898e61cd2b8ccd9f3f35dff101d241da2eacff9cc8d5123fe5a897efbeda4974b",
# ]


cts_hex = [
    "71fe1ace4389087266117cd7c98c4182851b3acff3b086e3f83f94d6eb05c4ba85d8e1fa14f11d1c3b568ff6cff5c09c5d67ef5c9c71b7eeb3d45a",
    "71fe1ace559a1e7266117cd7ce8745d7be2e74c3f0f68eeef57e8884e607debf81dfa0f012f95819681ae7f29fe4839b5175ef5e8760bef0b9d44b",
    "72fe069c51c81a20775928c7879d4fd2a93c3acff3f69fe5fe2e9493a303d9ea98c4e5b60ae40a146058e7c787fbd09a1474e25dc865b5e6af865d",
    "67e543885b9a5b2267177084cf8453ccb8633ad7fdb39de5b13f8a93a304d6bf8bc4f4ef5def110b6f56a3e186e2c68c1470ef5c9c2ffbd6a29157",
    "71fe029a148c1236320d7192878a59cfbc3a6ec5e7f68befb13196d6ea1ec4ea81d9e3fe50ea0f196d02a2f7cfe2c29c5577e35d8630baf6ea8046",
    "6ef914ce5989152b321a769ad79c42c7be6f6ad2fab19de1fc339d84f04ad3a589dfa0ff09ab0c196f13e7e780b4c097556ded57c871fbeea39346",
    "71fe1ace4389087266117cd7c4865bd2b93b7fd2b5a58ce9f4308c9ff01e97ab82cbf2ef5dfc101d6a56b3fb8ab4d08b4167ef5c9c30b8f0ab9745",
    "71fe029a148c1437615978d7c58854dbec2c75cde5a39be5e37e9b97ef0697a285dfa0f01cff101d764983f29bf500000000000000000000000000",
    "71fe1ace50875b31730d6ad7cb8640c7ec3c73d4e1bf81e7b13796d6e518d8a4988ceff05dff101d2415a8fe9fe1d79a4623eb5e8430bfe3b3d442",
    "71fe029a1483123c76597691878459cca9363ac4faf68ceffc2e8d82e61897b98fc5e5f809e20b0c7756b2e08aab83bc5560e25700000000000000",
    "71fe0680149d083b7c1e3996879a42d0a92e7780f6bf9fe8f42cd898e61cd2b8ccd9f3f35dff101d241da2eacff9cc8d5123fe5a897efbeda4974b",
]


# print("[")
# for ct in cts_hex:
#     print(f'"{ct}",')
# print("]")

ciphertexts = [bytes.fromhex(ct) for ct in cts_hex]


# for ct in ciphertexts:
#     print(ct)



def check_next_key_byte(idx, key, keychain):
    curr = keychain[idx]
    for i in range(len(key[idx])):
        if curr != key[idx][i]:
            keychain[idx] = key[idx][i]



def check_candidate_byte(pos, ct=ciphertexts):

    candidates = []
    for i in range(256):
        match = True
        for c in ct:
            if c[pos] == 0:
                continue
            res = chr(c[pos] ^ i)
            if res not in ALLOWED:
                match = False
                break
            elif pos == 0 and res not in string.ascii_uppercase:
                match = False
                break

        if match:
            candidates.append(i)

    return candidates


def print_xor_table_multi(cts, key, keychain):

    found = set()
    for i, k in enumerate(keychain):
        if len(k) == 1:
            found.add(i)

    n = min(min(len(ct) for ct in cts), len(key))

    sep = " "  # 3 spaces between columns
    colw = 2  # width of each column

    def hex2(x):
        return f"{x:02x}"

    def dec2(x):
        return f"{x:02d}"

    def is_printable_ascii(x):
        return 32 <= x <= 126

    # Index header (printed once)
    idx_row = sep.join(dec2(i) for i in range(n))

    for i, ct in enumerate(cts):
        # Ciphertext row
        print("idx - " + idx_row)
        ct_row = sep.join(hex2(ct[j]).upper() for j in range(n))
        print(f"ct{hex(i)[2:].upper()} - " + ct_row)

        # Key row
        k_row = []
        for j in range(n):
            res = key[j]
            # print(type(res), res)
            if j in found:
                cell = green(hex2(res).rjust(colw))
            else:
                cell = hex2(res).rjust(colw)

            k_row.append(cell)

        # print(k_row)
        key_row = sep.join(k_row)
        print(f"key - {key_row}")

        # Result row
        res_cells = []
        for j in range(n):
            x = ct[j] ^ key[j]
            if ct[j] == 0:
                break
            elif is_printable_ascii(x):
                if j in found:
                    cell = green(chr(x).rjust(colw))
                else:
                    cell = chr(x).rjust(colw)
            else:
                cell = hex2(x)

            res_cells.append(cell)

        res_row = sep.join(res_cells)
        print("res - " + res_row)

        if i != len(cts) - 1:
            print()  # blank line between CT blocks


def check_key_at_idx(ct, key, idx):
    for c in ct:
        res = c[idx] ^ key
        if chr(res) not in ALLOWED:
            print(f"{key} is not a valid key. {chr(res)}")
            return False

    return True


def get_key_byte(guess, pos, target_ct=-1, ct=ciphertexts):
    return ct[target_ct][pos] ^ ord(guess)


GREEN = "\033[32m"
RESET = "\033[0m"


def green(text):
    return f"{GREEN}{text}{RESET}"


def main():
    keychain = []
    key = []
    for i in range(len(ciphertexts[0])):
        k = check_candidate_byte(i)
        if not k:
            k = [0]
        key.append(k)
        keychain.append(k[0])
        print(f"key_byte_{i}: {k} - {len(k)}")

    print(keychain)

    # res = []
    # for c in ciphertexts:
    #     pt = []
    #     for i in range(len(keychain)):
    #         pt.append(chr(c[i] ^ keychain[i]))
    #     res.append(pt)

    # pt = []
    # for r in res:
    #     guess = "".join(r)
    #     pt.append(guess)

    # for p in pt:
    #     print(p)

    # idx = 0
    # guess = "W"
    # key_byte = get_key_byte(guess, idx)
    # if check_key_at_idx(ciphertexts, key_byte, idx):
    #     keychain[idx] = key_byte

    # idx = 1
    # guess = "h"
    # key_byte = get_key_byte(guess, idx)
    # if check_key_at_idx(ciphertexts, key_byte, idx):
    #     keychain[idx] = key_byte

    # idx = 9
    # guess = "g"
    # key_byte = get_key_byte(guess, idx)
    # if check_key_at_idx(ciphertexts, key_byte, idx):
    #     keychain[idx] = key_byte

    # idx = 13
    # guess = "m"
    # key_byte = get_key_byte(guess=guess, pos=idx, target_ct=-2)
    # if check_key_at_idx(ciphertexts, key_byte, idx):
    #     keychain[idx] = key_byte

    # idx = 14
    # guess = "o"
    # key_byte = get_key_byte(guess=guess, pos=idx, target_ct=-2)
    # if check_key_at_idx(ciphertexts, key_byte, idx):
    #     keychain[idx] = key_byte

    # idx = 18
    # guess = " "
    # key_byte = get_key_byte(guess=guess, pos=idx, target_ct=-2)
    # if check_key_at_idx(ciphertexts, key_byte, idx):
    #     keychain[idx] = key_byte

    print_xor_table_multi(ciphertexts, keychain, key)
    user_prompt(ciphertexts, keychain, key)


def user_prompt(ciphertexts, keychain, key):
    import string
    
    ALLOWED = set(string.ascii_letters + string.digits + " ,.?-")
    
    while True:
        # Step 1: Validate ciphertext number
        ct_input = input("Enter target ciphertext # (0-9) or q to quit: ")
        
        if ct_input == 'q':
            print("Exiting...")
            break
        
        if ct_input not in "0123456789":
            print("Invalid ciphertext #. Please enter 0-9.")
            continue
        
        ciphertext_num = int(ct_input)
        
        # Step 2: Validate target character
        while True:
            char_input = input("Enter target character (a-z, A-Z, 0-9, space, period, question mark, hyphen): ")
            
            if char_input not in ALLOWED:
                print("Invalid character. Allowed: {a-z, A-Z, 0-9, space (.), (,), (?), (-)}.")
                continue
            
            break
        
        # Step 3: Validate target index
        max_index = len(ciphertexts[ciphertext_num]) - 1
        
        while True:
            idx_input = input(f"Enter target index (0-{max_index}): ")
            
            # Check if numeric
            if not idx_input.isdigit():
                print(f"Invalid index. Must be numeric between 0 and {max_index}.")
                continue
            
            target_index = int(idx_input)
            
            # Check range
            if target_index < 0 or target_index > max_index:
                print(f"Invalid index. Must be numeric between 0 and {max_index}.")
                continue
            
            # Check first character constraint
            if target_index == 0 and char_input not in string.ascii_uppercase:
                print("First character (index 0) must be an uppercase letter.")
                continue
            
            break
        
        # Step 4: Process and update keychain
        key_byte = get_key_byte(guess=char_input, pos=target_index, target_ct=ciphertext_num)
        
        if check_key_at_idx(ciphertexts, key_byte, target_index):
            keychain[target_index] = key_byte
            print(f"✓ Guess accepted! Keychain[{target_index}] updated to {hex(key_byte)}.")
            print(f"{len(key[target_index])} candidate(s) remain at position {target_index}.\n")
            print_xor_table_multi(ciphertexts, keychain, key)
            print()
        else:
            print(f"✗ Guess rejected. Character '{char_input}' is invalid at index {target_index} for at least one ciphertext.")


if __name__ == "__main__":
    main()


# key =  269663ee34e87b52127919f7a7e936a2cc4f1aa095d6ef80915ef8f6836ab7caecac80967d8b78780476c793ef94a3ff34038a32e810db82caf42e
# target_pt =  "When using a stream cipher never use the key more than once"


key_byte_0: [32, 33, 34, 35, 36, 37, 38, 40, 43, 52, 54, 55, 61, 62, 63] - 15
key_byte_1: [136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 156, 157, 159, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 188, 189, 191] - 38
key_byte_2: [99, 109, 110, 114, 115] - 5
key_byte_3: [238, 241] - 2
key_byte_4: [52, 58] - 2
key_byte_5: [232] - 1
key_byte_6: [123] - 1
key_byte_7: [82] - 1
key_byte_8: [18] - 1
key_byte_9: [116, 117, 121] - 3
key_byte_10: [25] - 1
key_byte_11: [247] - 1
key_byte_12: [167] - 1
key_byte_13: [201, 203, 205, 206, 207, 222, 233, 235, 237, 238, 239, 254] - 12
key_byte_14: [3, 21, 22, 23, 35, 53, 54, 55] - 8
key_byte_15: [162, 189] - 2
key_byte_16: [204, 211, 221] - 3
key_byte_17: [79] - 1
key_byte_18: [5, 26] - 2
key_byte_19: [160] - 1
key_byte_20: [149] - 1
key_byte_21: [198, 199, 201, 214] - 4
key_byte_22: [200, 201, 202, 205, 207, 232, 233, 234, 237, 239] - 10
key_byte_23: [128, 130, 132, 134, 138, 139, 14